This project utilizes the PokeAPI (https://pokeapi.co/) to create a large database containing relvent information on Pokemon to be used for future projects. Note: This is not fully clean data. Working to wrangle further

Import the required Libraries

In [76]:
import requests
import pandas as pd

Create The Inital pd dataframes

In [77]:
pokedf1 = pokedf =  pd.DataFrame(columns = ['name','species', 'height', 'weight', 'abilities', 'types', 'stats'])
specdf1 = specdf = pd.DataFrame(columns = ['name','pokedex','gen','genus', 'capture_rate', 'color', 'shape', 'egg_groups', 'is_baby', 'is_legendary', 'is_mythical'])

Functions to update the dataframes

In [78]:
def update_pokedf(pokemon_info):
    #appending rows
    new_row = pokemon_info
    pokedf1.loc[len(pokedf1)] = new_row

    return None

def update_specdf(species_info):

    new_row = species_info
    specdf1.loc[len(specdf1)] = new_row

    return None

Get the pokemon info and append to df

In [79]:
def get_pokemon_info(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        pokemon_data = response.json()
        pokemon_info = {
            "name": pokemon_data["name"],
            "species": pokemon_data["species"]["name"],
            "height": pokemon_data["height"],
            "weight": pokemon_data["weight"],
            "abilities": [ability["ability"]["name"] for ability in pokemon_data["abilities"]],
            "types": [stats_data["type"]["name"] for stats_data in pokemon_data["types"]],
            "stats": [stats_data["base_stat"] for stats_data in pokemon_data["stats"]],
            #"hp": pokemon_data["stats"][0]["base_stat"],
            #"attack": pokemon_data["stats"][1]["base_stat"],
            #"defence": pokemon_data["stats"][2]["base_stat"],
            #"sp_attack": pokemon_data["stats"][3]["base_stat"],
            #"sp_defence": pokemon_data["stats"][4]["base_stat"],
            #"speed": pokemon_data["stats"][5]["base_stat"],
        }
        update_pokedf(pokemon_info)
        return pokemon_info
    else:
        return None

Get the species info and append to df

In [80]:
def get_species_info(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        species_data = response.json()
        species_info = {
            "name": species_data["name"],
            "pokedex": species_data["pokedex_numbers"][0]["entry_number"],
            "gen": species_data["generation"]["name"],
            #"genus": species_data["genera"]["genus"],
            "capture_rate": species_data["capture_rate"],
            "color": species_data["color"]["name"],
            #"shape": species_data["shape"]["name"],
            "egg_groups": [group["name"] for group in species_data["egg_groups"]],
            "is_baby": species_data["is_baby"],
            "is_legendary": species_data["is_legendary"],
            "is_mythical": species_data["is_mythical"],            
        }
        update_specdf(species_info)
        return species_info
    else:
        return None

Create the Pokemon df

In [81]:
#url = f"https://pokeapi.co/api/v2/pokemon?limit=100000&offset=0"    #Real URL
url = f"https://pokeapi.co/api/v2/pokemon?limit=50&offset=0"        #test URL
response = requests.get(url)

poke = response.json()

for pokemon in poke['results']:
    get_pokemon_info(pokemon['url'])

Create the Species df

In [82]:
#url = f"https://pokeapi.co/api/v2/pokemon-species?limit=100000&offset=0"   #Real URL
url = f"https://pokeapi.co/api/v2/pokemon-species?limit=50&offset=0"    #test URL
response = requests.get(url)

species = response.json()

for spec in species['results']:
    get_species_info(spec['url'])

#print(specdf1)

Function to edit generation format in data

In [83]:
def get_generation(generation):
    if generation == 'generation-i':
        return 1
    elif generation == 'generation-ii':
        return 2
    elif generation == 'generation-iii':
        return 3
    elif generation == 'generation-iv':
        return 4
    elif generation == 'generation-v':
        return 5
    elif generation == 'generation-vi':
        return 6
    elif generation == 'generation-vii':
        return 7
    elif generation == 'generation-viii':
        return 8
    elif generation == 'generation-ix':
        return 9
    else:
        return "NA"

Manipulating Dataframes -- 
Format Generation to numeric --- 
Join Pokemon/Species df --- 
Export to CSV

In [84]:
#Spread out Types/Abilities/Stats
pokedf1[['hp', 'attack', 'defence', 'sp_attack', 'sp_defence', 'speed']] = pd.DataFrame(pokedf1.stats.tolist(), index= pokedf1.index)
pokedf1[['type1','type2']] = pd.DataFrame(pokedf1.types.tolist(), index= pokedf1.index)
pokedf1[['ability1','ability2', 'hidden_ability']] = pd.DataFrame(pokedf1.abilities.tolist(), index= pokedf1.index)

#Drop Columns of lists
pokedf1 = pokedf1.drop("types", axis = 1)
pokedf1 = pokedf1.drop("abilities", axis = 1)
pokedf1 = pokedf1.drop("stats", axis = 1)

In [85]:
#Format Generation
specdf1["generation"] = specdf1["gen"].apply(get_generation)
specdf1 = specdf1.drop("gen", axis = 1)

specdf1[['egg_group1','egg_group2']] = pd.DataFrame(specdf1.egg_groups.tolist(), index= specdf1.index)
specdf1 = specdf1.drop("egg_groups", axis = 1)

In [86]:
df2 = pd.merge(pokedf1, specdf1, how="left", left_on='species', right_on='name')
df2 = df2.drop("name_y", axis = 1)
df2 = df2.rename(columns={'name_x': 'name'})

In [98]:
df2 = df2.sort_values(by=['pokedex', 'name'])
df2.to_csv("pokemon_data.csv", index = False)

Preview dataframes

In [88]:
pokedf1.head()

,name,species,height,weight,hp,attack,defence,sp_attack,sp_defence,speed,type1,type2,ability1,ability2,hidden_ability
0,bulbasaur,bulbasaur,7,69,45,49,49,65,65,45,grass,poison,overgrow,chlorophyll,None
1,ivysaur,ivysaur,10,130,60,62,63,80,80,60,grass,poison,overgrow,chlorophyll,None
2,venusaur,venusaur,20,1000,80,82,83,100,100,80,grass,poison,overgrow,chlorophyll,None
3,charmander,charmander,6,85,39,52,43,60,50,65,fire,None,blaze,solar-power,None
4,charmeleon,charmeleon,11,190,58,64,58,80,65,80,fire,None,blaze,solar-power,None


In [89]:
specdf1.head()

,name,pokedex,genus,capture_rate,color,shape,is_baby,is_legendary,is_mythical,generation,egg_group1,egg_group2
0,bulbasaur,1,NaN,45,green,NaN,False,False,False,1,monster,plant
1,ivysaur,2,NaN,45,green,NaN,False,False,False,1,monster,plant
2,venusaur,3,NaN,45,green,NaN,False,False,False,1,monster,plant
3,charmander,4,NaN,45,red,NaN,False,False,False,1,monster,dragon
4,charmeleon,5,NaN,45,red,NaN,False,False,False,1,monster,dragon


In [97]:
df2.head()

,name,species,height,weight,hp,attack,defence,sp_attack,sp_defence,speed,...,genus,capture_rate,color,shape,is_baby,is_legendary,is_mythical,generation,egg_group1,egg_group2
0,bulbasaur,bulbasaur,7,69,45,49,49,65,65,45,...,NaN,45,green,NaN,False,False,False,1,monster,plant
1,ivysaur,ivysaur,10,130,60,62,63,80,80,60,...,NaN,45,green,NaN,False,False,False,1,monster,plant
2,venusaur,venusaur,20,1000,80,82,83,100,100,80,...,NaN,45,green,NaN,False,False,False,1,monster,plant
3,charmander,charmander,6,85,39,52,43,60,50,65,...,NaN,45,red,NaN,False,False,False,1,monster,dragon
4,charmeleon,charmeleon,11,190,58,64,58,80,65,80,...,NaN,45,red,NaN,False,False,False,1,monster,dragon
